<a href="https://colab.research.google.com/github/Dr-Carlos-Villasenor/TopicosAvanzadosComputacion2/blob/master/DL_L09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tópicos Avanzados de Computación II
## Dr. Carlos Villaseñor
## Lección 9 - Redes Neuronales convolucionales II
### Practica en Aprendizaje de transferencia y aumentación de datos

Importar bibliotecas



In [1]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import mobilenet_v2
from keras.applications import mobilenet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import adam_v2

# Importamos unos paquetes de sklearn para comprobar el modelo
from sklearn.metrics import classification_report, confusion_matrix

Importar MobileNet base pero descartaremos las últimas capa

In [2]:
base_model= mobilenet_v2.MobileNetV2(weights='imagenet',include_top=False)
# base_model= mobilenet.MobileNet(weights='imagenet',include_top=False)
print(base_model.summary())

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 Conv1 (Conv2D)                 (None, None, None,   864         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, None, None,   128         ['Conv1[0][0]']                  
                                32)                                            

Crear nueva Red Neuronal

In [3]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(32,activation='relu')(x)
x=Dense(16,activation='relu')(x)
x=Dense(8,activation='relu')(x)
preds=Dense(5, activation='softmax')(x)

Pegar Redes Neuronales

In [4]:
model=Model(inputs=base_model.input, outputs=preds)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 Conv1 (Conv2D)                 (None, None, None,   864         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, None, None,   128         ['Conv1[0][0]']                  
                                32)                                                           

Definir que capas se entrenan

In [5]:
for layer in model.layers[:10]:
    layer.trainable=False
for layer in model.layers[10:]:
    layer.trainable=True

Cargar datos nuevos

- Carge el archivo 'train.zip' en la pestaña archivos en el lateral izquierdo
- Corra la siguiente linea para descomprimir los archivos

In [ ]:
!unzip train.zip 

Crear generador desde carpeta

In [6]:
train_datagen=ImageDataGenerator(zoom_range=0.2, horizontal_flip=True,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 preprocessing_function=preprocess_input)

train_generator=train_datagen.flow_from_directory('./DataSets/train',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)



Found 362 images belonging to 5 classes.


In [7]:
test_datagen=ImageDataGenerator(zoom_range=0.2, horizontal_flip=True,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 preprocessing_function=preprocess_input)

test_generator=test_datagen.flow_from_directory('./DataSets/test',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 42 images belonging to 5 classes.


Compilar Modelo y entrenar desde carpeta

In [8]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',
              metrics=['accuracy'])

step_size_train=train_generator.n//train_generator.batch_size

model.fit(x=train_generator,
                    steps_per_epoch=step_size_train,
                    epochs=6)

Epoch 1/6
 6/11 [===============>..............] - ETA: 28s - loss: 1.2466 - accuracy: 0.4062

c:\users\aleja\pycharmprojects\deeplearning\vc-env\lib\site-packages\PIL\Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


11/11 [==============================] - 76s 5s/step - loss: 1.0024 - accuracy: 0.5000
Epoch 2/6
11/11 [==============================] - 54s 5s/step - loss: 0.7130 - accuracy: 0.6667
Epoch 3/6
11/11 [==============================] - 58s 5s/step - loss: 0.5492 - accuracy: 0.8182
Epoch 4/6
11/11 [==============================] - 72s 6s/step - loss: 0.3995 - accuracy: 0.8455
Epoch 5/6
11/11 [==============================] - 62s 5s/step - loss: 0.4185 - accuracy: 0.8273
Epoch 6/6
11/11 [==============================] - 81s 7s/step - loss: 0.4245 - accuracy: 0.8576


In [9]:
model.evaluate(x=train_generator, steps = step_size_train)

11/11 [==============================] - 46s 4s/step - loss: 2.7128 - accuracy: 0.4915


[2.7127931118011475, 0.49147728085517883]

In [11]:
step_size_test=test_generator.n//test_generator.batch_size
model.evaluate(x=test_generator, steps = step_size_test)

1/1 [==============================] - 4s 4s/step - loss: 8.5624 - accuracy: 0.3750


[8.56240463256836, 0.375]

In [11]:
Y_pred = model.predict(x=test_generator)
y_pred = np.argmax(Y_pred, axis=1)

print('Matriz de confusión:\n', confusion_matrix(test_generator.classes, y_pred))

print('Classification Report')
target_names = ['Birds', 'Cats', 'Dogs', 'Horses', 'Wolfs']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

Matriz de confusión:
 [[0 2 4 1 1]
 [5 0 0 0 3]
 [3 1 3 1 1]
 [0 2 3 1 2]
 [3 1 1 0 4]]
Classification Report
              precision    recall  f1-score   support

       Birds       0.00      0.00      0.00         8
        Cats       0.00      0.00      0.00         8
        Dogs       0.27      0.33      0.30         9
      Horses       0.33      0.12      0.18         8
       Wolfs       0.36      0.44      0.40         9

    accuracy                           0.19        42
   macro avg       0.19      0.18      0.18        42
weighted avg       0.20      0.19      0.18        42

